# Gentrification
This notebook implements the two tests described in Governing Magazine's [Gentrification Report Methodology](http://www.governing.com/gov-data/gentrification-report-methodology.html).

In [157]:
from collections import defaultdict
import geopandas as gpd
import pandas as pd

In [161]:
from configparser import RawConfigParser
import os.path
import uscensus

First load the geocoded addresses and extract the distinct census states and counties (we will retrieve data for all tracts)

In [158]:
df = pd.read_csv('geocoded_addresses.csv', dtype={
    'Geo.FIPS.State': str,
    'Geo.FIPS.County': str,
    'Geo.Tract': str,
    'Geo.Block': str,
})

In [159]:
cols = ['Geo.FIPS.State',
        'Geo.FIPS.County',
       ]
geos = pd.DataFrame(
    list(set(df[cols].dropna().apply(tuple, axis=1))),
    columns=cols
)

In [160]:
geos.shape

(28, 2)

In [162]:
rcp = RawConfigParser()

In [163]:
rcp.read(os.path.expanduser('~/.census'))

['C:\\cygwin64\\home\\nkrishna/.census']

In [164]:
key = rcp.get('census', 'api_key')

In [165]:
cache = uscensus.util.SqlAlchemyCache('sqlite://')

In [166]:
di11 = uscensus.DiscoveryInterface(key, cache, vintage=2011)

Indexing metadata
Done indexing metadata


Start with the 2011 ACS 5-year estimates

In [167]:
acs5_2011 = di11['2011/acs5']

Incrementally search to find the right variables

In [168]:
acs5_2011.searchVariables('concept:B01003')

[('B01003_001M',
  {'concept': 'B01003.  Total Population',
   'group': 'N/A',
   'label': 'Margin Of Error For!!Total',
   'limit': 0,
   'validValues': []}),
 ('B01003_001E',
  {'concept': 'B01003.  Total Population',
   'group': 'N/A',
   'label': 'Total',
   'limit': 0,
   'validValues': []})]

In [187]:
stats_11 = None
fields_11 = {
    'B01003_001E': 'Total.Population',
    'B19013_001E': 'Median.Household.Income',
    'B25077_001E': 'Median.Home.Value',
    'B16010_041E': 'Bachelors.Or.Higher',
}

Fetch the fields for each tract in each county

In [188]:
for st, cty in geos.itertuples(index=False):
    stats_cty = acs5_2011(
        fields.keys(),
        geo_for={'tract': '*'},
        geo_in={'state': st,
         'county': cty,
        })
    if stats_11 is None:
        stats_11 = stats_cty
    else:
        stats_11 = stats_11.append(stats_cty)

> The first test found that a tract was eligible to gentrify if it met the following criteria:
>
> 1. The tract had a population of at least 500 residents at the beginning and end of a decade and was  located within a central city. 
> 2. The tract’s median household income was in the bottom 40th percentile when compared to all tracts within its metro area at the beginning of the decade.
> 3. The tract’s median home value was in the bottom 40th percentile when compared to all tracts within its metro area at the beginning of the decade.


In [189]:
stats_11.rename(columns=fields_11, inplace=True)

stats_11.set_index(['state','county','tract'], inplace=True)

stats_11.sort_values('Median.Household.Income', inplace=True)
stats_11['Income.Rank'] = range(1, stats_11.shape[0] + 1)
stats_11['Income.Percentile'] = stats_11['Income.Rank'] / len(stats_11['Income.Rank'])

stats_11.sort_values('Median.Home.Value', inplace=True)
stats_11['Home.Value.Rank'] = range(1, stats_11.shape[0] + 1)
stats_11['Home.Value.Percentile'] = stats_11['Home.Value.Rank'] / len(stats_11['Home.Value.Rank'])

In [193]:
stats_11.sort_index()

Total.Population Bachelors.Or.Higher  \
state county tract                                         
05    107    480100             3247                 480   
             480200             6206                 557   
             480300             6018                 272   
             480400             1882                 108   
             480500             2655                 179   
             480600             2047                  87   
12    111    380100              767                  40   
             380200             3664                 132   
             380300             6657                 337   
             380400             5662                 229   
             380500             6582                 588   
             380600             4163                 152   
             380700             7667                 696   
             380800             3305                 278   
             380901             2450                 181   
             380902             5544                 358   
             381000             1427                 222   
             381101             7433                 465   
             381102             4260                 544   
             381204             2670                1209   
             381300             3381                1022   
             381401             4612                 490   
             381402             5653                 445   
             381502            10669                1360   
             381503            10042                 851   
             381601             6415                 488   
             381602             2974                 248   
             381603             6289                1086   
             381701             3163                1318   
             381702             2083                 417   
...                              ...                 ...   
53    053    073117             4113                 551   
             073118             4673                 514   
             073119             2884                 373   
             073120             3582                 298   
             073121             5505                1003   
             073122             4492                 671   
             073123             5097                1205   
             073124             3078                 394   
             073125             8973                1114   
             073126             4507                 631   
             073200             5854                 859   
             073301             5652                 569   
             073302             3838                 556   
             073404             6346                 907   
             073405             3563                 282   
             073406             4368                 714   
             073407             3599                 343   
             073408             4195                 556   
             073500             6717                1325   
             940001             2791                 980   
             940002             3802                 461   
             940003             6567                 909   
             940004             6246                 719   
             940005             5923                 765   
             940006             1882                 120   
             940007             3345                 270   
             940008             6402                1617   
             940009             3076                 624   
             940010             3945                 259   
             940011             4165                1554   

                    Median.Household.Income Median.Home.Value  Income.Rank  \
state county tract                                                           
05    107    480100                   32891             85300         1088   
     

And the 2015 ACS 5

In [ ]:
di15 = uscensus.DiscoveryInterface(key, cache, vintage=2015)

In [217]:
acs5_2015 = di15.search('American Community Survey 5-year estimates')[0]

In [219]:
for f in fields_11:
    print(acs5_2015.searchVariables('variable:'+f))

[('B01003_001E', {'label': 'Total', 'concept': 'B01003.  Total Population', 'group': 'N/A', 'limit': 0, 'validValues': [], 'predicateType': 'int'})]
[('B16010_041E', {'label': "Bachelor's degree or higher:", 'concept': 'B16010.  EDUCATIONAL ATTAINMENT AND EMPLOYMENT STATUS BY LANGUAGE SPOKEN AT HOME FOR THE POPULATION 25 YEARS AND OVER', 'group': 'N/A', 'limit': 0, 'validValues': [], 'predicateType': 'int'})]
[('B19013_001E', {'label': 'Median household income in the past 12 months (in 2015 Inflation-adjusted dollars)', 'concept': 'B19013. Median Household Income in the Past 12 Months (in 2015 Inflation-Adjusted Dollars)', 'group': 'N/A', 'limit': 0, 'validValues': [], 'predicateType': 'int'})]
[('B25077_001E', {'label': 'Median value (dollars)', 'concept': 'B25077.  Median Value (Dollars) for Owner-Occupied Housing Units', 'group': 'N/A', 'limit': 0, 'validValues': [], 'predicateType': 'int'})]


In [221]:
fields_15 = fields_11
stats_15 = None

In [222]:
for st, cty in geos.itertuples(index=False):
    stats_cty = acs5_2015(
        fields.keys(),
        geo_for={'tract': '*'},
        geo_in={'state': st,
         'county': cty,
        })
    if stats_15 is None:
        stats_15 = stats_cty
    else:
        stats_15 = stats_15.append(stats_cty)

> For a second test, gentrification-eligible tracts were determined to have gentrified over a time period if they met the following criteria:
> 
> 1. An increase in a tract's educational attainment, as measured by the percentage of residents age 25 and over holding bachelor’s degrees, was in the top third percentile of all tracts within a metro area.
> 2. A tract’s median home value increased when adjusted for inflation.
> 3. The percentage increase in a tract’s inflation-adjusted median home value was in the top third percentile of all tracts within a metro area.
>
> Census tracts with missing data for any one of these measures were excluded and considered not eligible to gentrify.

In [223]:
stats_15.rename(columns=fields_15, inplace=True)

stats_15.set_index(['state','county','tract'], inplace=True)

stats_15.sort_values('Median.Household.Income', inplace=True)
stats_15['Income.Rank'] = range(1, stats_15.shape[0] + 1)
stats_15['Income.Percentile'] = stats_15['Income.Rank'] / len(stats_15['Income.Rank'])

stats_15.sort_values('Median.Home.Value', inplace=True)
stats_15['Home.Value.Rank'] = range(1, stats_15.shape[0] + 1)
stats_15['Home.Value.Percentile'] = stats_15['Home.Value.Rank'] / len(stats_15['Home.Value.Rank'])

In [225]:
stats = stats_11.join(stats_15, lsuffix='.2011', rsuffix='.2015')

In [237]:
pop500_2011 = stats['Total.Population.2011'].astype(int) >= 500
pop500_2015 = stats['Total.Population.2015'].astype(int) >= 500
income_bot_40 = stats['Income.Percentile.2011'] < 0.40
home_value_bot_40 = stats['Home.Value.Percentile.2011'] < 0.40

In [238]:
gent_eligible = pop500_2011 & pop500_2015 & income_bot_40 & home_value_bot_40

In [239]:
gent_eligible.sum()

387

In [240]:
stats['Gentrification.Eligible'] = gent_eligible

In [246]:
stats['Ed.Attainment.Increase'] = (
    (stats['Bachelors.Or.Higher.2015'].astype(int) /
     stats['Total.Population.2015'].astype(int)) -
    (stats['Bachelors.Or.Higher.2011'].astype(int) /
     stats['Total.Population.2011'].astype(int)))

In [248]:
stats.sort_values('Ed.Attainment.Increase', inplace=True)

In [249]:
stats['Ed.Attainment.Increase.Rank'] = range(1, stats.shape[0] + 1)
stats['Ed.Attainment.Increase.Percentile'] = stats['Ed.Attainment.Increase.Rank'] / len(stats['Ed.Attainment.Increase.Rank'])

In [250]:
top_third_ed_att_inc = stats['Ed.Attainment.Increase.Percentile'] >= 2/3

For the inflation adjustment rate, I used the BLS's calculator:
https://data.bls.gov/cgi-bin/cpicalc.pl?cost1=100&year1=201101&year2=201501

In [258]:
inflation = 1.0612
has_val = stats.dropna(subset=['Median.Home.Value.2011', 'Median.Home.Value.2015'])
home_val_change = (has_val['Median.Home.Value.2015'].astype(int) -
                   has_val['Median.Home.Value.2011'].astype(int) * inflation)

In [259]:
stats['Home.Value.Increased'] = home_val_change > 0
stats['Home.Value.Change'] = home_val_change
stats.sort_values('Home.Value.Change', inplace=True)
stats['Home.Value.Change.Rank'] = range(1, stats.shape[0] + 1)
stats['Home.Value.Change.Percentile'] = stats['Home.Value.Change.Rank'] / len(stats['Home.Value.Change.Rank'])

In [266]:
gentrified = (stats['Gentrification.Eligible'] &
              top_third_ed_att_inc &
              stats['Home.Value.Increased'] &
              stats['Home.Value.Change.Percentile'] >= 2/3)

In [268]:
stats['Gentrified'] = gentrified

In [270]:
stats[stats['Gentrified']]

Total.Population.2011 Bachelors.Or.Higher.2011  \
state county tract                                                   
48    201    342800                  8692                     1542   
             541203                  2413                      834   
             340900                  5083                      538   
      029    121403                  5589                      307   
      439    113407                  6452                      382   
      113    000405                  2538                      113   
      029    181725                  5218                      341   
      453    002108                  3814                      419   
      439    113929                  8696                     2079   
      453    001805                  4547                      405   
      029    121903                  5001                     1766   
      453    002422                  5270                      752   
      201    554903                 11293                     3014   
      113    016617                  4537                     1535   
      439    106512                  5014                      694   
      113    006900                  3474                       87   
      453    002307                  5567                      834   
      201    310300                  5327                      597   
             551702                  3987                     1224   
             422600                  6490                     1024   
      339    691302                  5402                     1105   
      201    250800                  8003                     2180   
      029    110300                  3153                      404   
      201    554402                 12390                     3381   
             421101                  3646                      440   
      029    181506                  4422                      506   
      441    012000                  2081                      848   
      167    725800                  3498                      359   
      029    191304                  2792                      270   
      453    001811                  3032                      300   
...                                   ...                      ...   
             002304                  4270                      514   
      201    520400                  3476                       66   
      113    013105                  6431                      927   
      453    002318                  5714                      498   
      113    001600                  4143                      965   
      453    000901                  1860                      412   
             000804                  2528                      525   
             000902                  5348                      717   
      215    024106                 11329                     1271   
      453    001812                  7381                      500   
             002109                  3171                      418   
      029    120701                  5233                      868   
      113    000800                  6399                      681   
      453    002111                  4365                      335   
             002005                  4522                     1001   
             000402                  2471                     1116   
             000801                  1394                      127   
      439    113116                  4060                      662   
             123400                  2416                      284   
      167    724101                  1787                      225   
      339    691301                  5243                     1987   
      453    002315                  3336                      299   
      215    021201                  2800                      481   
      201    510100                  1815                      401   
      113    020600           

In [271]:
stats[['Gentrification.Eligible', 'Gentrified']].to_csv('gentrification.csv')